In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import pickle
from sklearn.ensemble import RandomForestClassifier


diabetes_dataset = pd.read_csv("../datasets/diabetes.csv")


x = diabetes_dataset.drop(columns=['Outcome'], axis=1)
y = diabetes_dataset['Outcome']


scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)


x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)


rf_model = RandomForestClassifier(random_state=42)


param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [None, 10, 20, 30],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}


pipeline = Pipeline([
    ('oversample', SMOTE(random_state=42)),
    ('undersample', RandomUnderSampler(random_state=42)),
    ('model', rf_model)
])


grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(x_train, y_train)


best_rf_model = grid_search.best_estimator_


cv_scores = cross_val_score(best_rf_model, x_train, y_train, cv=5)
print(f"Random Forest Cross-Validation Accuracy: {np.mean(cv_scores) * 100:.2f}%")


best_rf_model.fit(x_train, y_train)


train_predictions = best_rf_model.predict(x_train)
train_accuracy = accuracy_score(y_train, train_predictions)


test_predictions = best_rf_model.predict(x_test)
test_accuracy = accuracy_score(y_test, test_predictions)


print(f"Random Forest Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Random Forest Test Accuracy: {test_accuracy * 100:.2f}%")


filename = 'diabetes_model.sav'
pickle.dump(best_rf_model, open(filename, 'wb'))

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Random Forest Cross-Validation Accuracy: 77.53%
Random Forest Training Accuracy: 92.51%
Random Forest Test Accuracy: 75.32%
